<a href="https://colab.research.google.com/github/Eddiebee/AI-Craft/blob/main/Prompt_Engineering_Techniques_with_Cohere.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prompt Engineering Best Practices

In this notebook I attempt to apply several Prompt Engineering Techniques to get the best results from an LLM.

Cheers!

We first import the necessary libraries:

In [ ]:
!pip install cohere -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00


In [ ]:
import cohere
import os
import IPython


In [ ]:
# API Config
from google.colab import userdata
COHERE_API_KEY = userdata.get('COHERE_TRIAL_KEY')

In [ ]:
co = cohere.Client(COHERE_API_KEY)

In [ ]:
# completion function
def get_completion(message, preamble=None, model="command-r", temperature=0, max_tokens=300):
  response = co.chat(model=model,
                     message=message,
                     preamble=preamble,
                     temperature=temperature,
                     max_tokens=max_tokens)

  return response.text


# Be Clear and Specific

It is great to keep the instruuctions for the model clear and specific and avoid ambiguous instructions.

In [ ]:
global_trending_movies = ["The Suicide Squad", "No Time to Die", "Dune",
                          "Spider-Man: No Way Home", "The French Dispatch",
                          "Black Widow", "Eternals", "The Matrix Resurrections",
                          "West Side Story", "The Many Saints of Newark"]

preamble = """
Your task is to recommend movies to a customer.

You are responsible to recommend a movie from the top global trending movies from {global_trending_movies}.

You should refrain from asking users for their preferences and avoid asking for personal information.

If you don't have a movie to recommend or don't know the user interests, you should respond "Sorry, couldn't find a movie to recommend today.".
"""

message = """
Please recommend a movie based on my interests.
"""

response = get_completion(message, preamble.format(
    global_trending_movies=global_trending_movies))
print(response)

Sorry, couldn't find a movie to recommend today. However, some of the latest and trending movies include "The Suicide Squad," an action-packed film, perfect for those who enjoy high-octane adventures. If you're a fan of thrillers and spy stories, "No Time to Die" might be a great choice. It's the latest installment in the James Bond franchise. 

For sci-fi lovers, the epic "Dune" could be an excellent pick, while "Spider-Man: No Way Home" is a delightful option for those who enjoy superhero movies with a twist. "The French Dispatch" is a charming and stylish comedy-drama, perfect for fans of quirky storytelling. 

If you're in the mood for some high-action adventure, the long-awaited "Black Widow" could be a fantastic option. Another fantasy adventure awaits in "Eternals," a fresh tale of immortal heroes. 

For fans of classic films, the iconic "The Matrix Resurrections" offers a thrilling return to the groundbreaking series. Another musical masterpiece is "West Side Story," a modern t

In the example below the customer provides some context for movies of interest to them.

In [ ]:
global_trending_movies = ["The Suicide Squad", "No Time to Die", "Dune",
                          "Spider-Man: No Way Home", "The French Dispatch",
                          "Black Widow", "Eternals", "The Matrix Resurrections",
                          "West Side Story", "The Many Saints of Newark"]

preamble = """
Your task is to recommend movies to a customer.

You are responsible to recommend a movie from the top global trending movies from {global_trending_movies}.

You should refrain from asking users for their preferences and avoid asking for personal information.

If you don't have a movie to recommend or don't know the user interests, you should respond "Sorry, couldn't find a movie to recommend today.".
"""

message = """
Please recommend a movie based on my interests. I love super-hero movies.
"""

response = get_completion(message, preamble.format(
    global_trending_movies=global_trending_movies))
print(response)

Based on your interest in superhero movies, I'd recommend the movie "Spider-Man: No Way Home." This movie is a thrilling addition to the Spider-Man franchise, with a fresh take on the classic character. The film explores the concept of the multiverse, leading to unexpected and exciting twists and turns. It's a fun and action-packed adventure that will keep you engaged from start to finish. With a great blend of humor, emotion, and thrilling action sequences, "Spider-Man: No Way Home" is a delightful treat for all fans of superhero cinema. 

If you want an even deeper dive into the superhero world, you might also enjoy "The Suicide Squad," a unique take on the traditional superhero film with a compelling story and diverse characters, or "Black Widow," which explores the backstory of this beloved Marvel character. 

Let me know if you would like to know more about any of these movies or if you have other interests I can take into account to provide a more tailored recommendation!


# Make use of Delimiters

This will help the language model understand the essence of the text in those sections and make more sense of them.

In [ ]:
prompt = """
Convert the following code block in the #### <code> #### section to Python:

####
strings2.push("one")
strings2.push("two")
strings2.push("THREE")
strings2.push("4")
####
"""

message = prompt

print(get_completion(message))

```python
strings2 = ["one", "two", "THREE", "4"]
```


# Specify the Output format

Let's say we need the output of the model to be in a specific format, then it makes so much sense to explicitly state this to the model. Yeah!

In [ ]:
prompt = """
Your task is: given a product description, return the requested information in the section delimited by ### ###. Format the output as a JSON object.

Product Description: Introducing the Nike Air Max 270 React: a comfortable and stylish sneaker that combines two of Nike's best technologies. \n
With a sleek black design and a unique bubble sole, these shoes are perfect for everyday wear.

###
product_name: the name of the product
product_bran: the name of the brand (if any)
###
"""

message = prompt

print(get_completion(message))

```json
{
  "product_name": "Nike Air Max 270 React",
  "product_brand": "Nike"
}
```


# Step-wise reasoning

To ensure the model pays attention to it's decision making process, we can explicitly tell it to break down and detail it's process step-by-step.

In [ ]:
prompt = """The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.

Solve by breaking the problem into steps. First, identify the odd numbers, add them, and indicate whether the result is odd or even."""

message = prompt

print(get_completion(message))

Let's solve this in steps, beginning by identifying the odd numbers:

1. 15 - Odd
2. 32 - Even
3. 5 - Odd
4. 13 - Odd
5. 82 - Even
6. 7 - Odd
7. 1 - Odd

Now, let's add up all the odd numbers:

15 + 5 + 13 + 7 + 1 = 34

So, the sum of all the odd numbers in this group is 34, which is even. Therefore, the solution to the problem is that the even sum of the odd numbers is 34.


In [ ]:
prompt = """The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.

Solve by breaking the problem into steps.
Show the steps taken, thanks!
"""

message = prompt

print(get_completion(message))

Let's solve this problem step by step:

1: Identify the numbers that are odd: 15, 5, 13, 7, and 1
2: Begin by adding the first two odd numbers: 15 + 5 = 20
3: Continue adding the next odd number to the running total: 20 + 13 = 33
4: Now add the remaining odd numbers: 33 + 7 + 1 = 41

Therefore the sum of the odd numbers in the given group is 41, which is indeed even.


In [ ]:
prompt = """The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.

Solve by breaking the problem into steps.
Tell us if the sum is actually even or not.
"""

message = prompt

print(get_completion(message))

Let's solve this step by step:
1. Identify the odd numbers in the given list: 15, 5, 13, 7, and 1 are odd. 
2. Add the first two: 15 + 5 = 20
3. Now add the next one: 20 + 13 = 33
4. Continue this process: 33 + 7 = 40
5. Finally, add the last number: 40 + 1 = 41.

So, the sum of all the odd numbers in the given list is 41, which is odd.


In [ ]:
prompt = """The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.

Solve by breaking the problem into steps.
Tell us if the sum is actually even or not.
"""

message = prompt

print(get_completion(message, model="command-r-plus"))

Step 1: Identify the odd numbers in the group: 15, 5, 13, 7, 1.

Step 2: Add these odd numbers together: 
15 + 5 + 13 + 7 + 1 = 41

Step 3: Determine if the sum is even or odd. In this case, 41 is an odd number. 

So, the statement is false. The sum of the odd numbers in this group is an odd number, not an even number.
